In [ ]:
# The below code is a quantitative trading brain child of mine that I have spent many hours thinking about and developing.
# It was also used in a Rutgers University Quantitative Finance Competition sponsored by SIG and allowed my team and I to become finalists.

# The general idea of the algorithm is to utilize machine learning models trained upon quarterly and yearly(unusual, I know) financial statements
# and their resulting price changes to create prediction of the price change over the next quarter. If the prediction is significant enough, the 
# company is added to a running list of companies on a "watch list". If a company on the watch list strays far enough from the predicted 
# course(2 standard devs) the algorithm predicts a reversion to the expected trend. The farther away from the expectation it is, the more 
# allocation the company recieves until a predetermined stop loss point.

# Note: The below code works only on a platform called Quant Connect which is a very useful tool that allows quantitative finance developers to access
# vast amounts of data and a useful platform to test and implement code. Given this, some code may be hard to follow as it utilizes the unique
# yet useful syntax native to the platform.

In [ ]:
# This is the Neural Networks code for Quant Connect:
from AlgorithmImports import *
from datetime import timedelta
from QuantConnect.Algorithm import QCAlgorithm
from QuantConnect.Orders import OrderStatus
from QuantConnect.Securities.Equity import Equity
from QuantConnect import Resolution
from QuantConnect.Orders import MarketOrder
from sklearn.linear_model import LinearRegression
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import keras
from keras import layers
from sklearn.model_selection import train_test_split
from QuantConnect.Data.UniverseSelection import * 
from Selection.FundamentalUniverseSelectionModel import FundamentalUniverseSelectionModel
import QuantConnect.Algorithm.Framework.Alphas
import statistics
from QuantConnect.DataSource import *

Symbols = []
Predictions = []
symbs = []

class NNAlphaModel(AlphaModel):
    def __init__(self, algorithm):
        self.qb = QuantBook()
        self.algorithm = algorithm
        self.industriesList = self.GetIndustries(algorithm)
        # algorithm.Debug("industriesList: "+str(self.industriesList))
        self.algorithm = algorithm
        self.sym = self.algorithm.AddIndex("SPX", Resolution.Daily).Symbol

    def Update(self, algorithm, data):
        insights = []
        for i in range(0,len(Symbols)-1):
            symbol = Symbols[i]
            predict = self.GetPrediction(symbol)
            if predict:
                # self.algorithm.Debug(f"This is the prediction for {symbol}: "+str(predict))
                Predictions[i] = predict

        if Symbols:
            for i in range (0,len(Symbols)-1):
                # Creates an array of insights for the computer to score
                # Insight.Price(symbol, period, direction, magnitude=None, confidence=None, sourceModel=None, weight=None, tag='') 
                insights.append(Insight(symbs[i], timedelta(days=90), InsightType.Price, InsightDirection.Up, Predictions[i], None))
            # self.algorithm.Debug("These are insights "+str(insights))
            return insights
        else:
            return None

    def OnSecuritiesChanged(self, algorithm: QCAlgorithm, changes: SecurityChanges):
        for symbol in changes.RemovedSecurities:
            for i in range(0,len(Symbols)-1):
                if symbol == Symbols[i]:
                    del Symbols[i]
                    symbs.remove(symbol.Symbol)
                    del Predictions[i]
                
        # self.algorithm.Debug("Symbols after removing securities on change: "+str(Symbols))
        for symbol in changes.AddedSecurities:
            symbs.append(symbol.Symbol)
            Symbols.append(symbol)
            Predictions.append(0)
        # self.algorithm.Debug("Symbols after adding securities on change"+str(Symbols))
        self.industriesList = self.GetIndustries(algorithm)

    def GetIndustries(self, algorithm):
        industries = []
        for symbol in Symbols:
            put = False
            SecCode = symbol.Fundamentals.AssetClassification.MorningstarSectorCode
            for group in industries:
                # self.algorithm.Debug("Group is "+str(group))
                if str(group[0]) == str(SecCode):
                    group.append(str(symbol))
                    # self.algorithm.Debug("Appended Group is: "+str(group))
                    put = True
                    break
            if not put:
                # self.algorithm.Debug("Put is false RN")
                industries.append([str(SecCode), str(symbol)])

        return industries

    def GetPrediction(self, symbol):
        if symbol.Fundamentals.HasFundamentalData:
            earningsDate = symbol.Fundamentals.EarningReports.FileDate.Value
            # self.algorithm.Debug(f"This is the earnings FileDate for {symbol}: {earningsDate}")

            if earningsDate == self.algorithm.Time:
                if symbol.Fundamentals.AssetClassification.MorningstarSectorCode is not None:
                    industry = symbol.Fundamentals.AssetClassification.MorningstarSectorCode
                    prediction = self.CalculateYpred(symbol, industry)
                    if prediction:
                        return prediction
                        self.algorithm.Debug(f"Prediction found for {str(symbol)}")

                    else:
                        return None
                else:
                    self.algorithm.Debug(f"No Sector Code found for {str(symbol)}")
                    return None
        else: 
            self.algorithm.Debug(f"{str(symbol)} does not have fundamental data.")
            self.algorithm.Log(f"{str(symbol)} does not have fundamental data.")
            return None

    def CalculateYpred(self, symbol, industry):
        industry_data = []
        a=0
        NowDate = self.algorithm.Time
        companiesOfIndustry = []
        prediction = []
        daysOfHistory = 36000 #2 years(adjust as needed)
        for indust in self.industriesList:
            if str(indust[0]) == str(industry):
                companiesOfIndustry = indust[1::]
                # self.algorithm.Debug(f"Companies of {symbol}'s industry({industry}) are {companiesOfIndustry}")
        for tick in companiesOfIndustry:
            symbol = tick
            for symb in Symbols:
                if str(tick) == str(symb):
                    tick = symb.Symbol
            hist_data = self.qb.History(Fundamental, symbol, NowDate - timedelta(days=daysOfHistory), NowDate) #change timedelta as needed(its small for backtesting purposes)
            # self.algorithm.Debug("length of Hist_data: "+str(len(hist_data)))
            if not hist_data.empty:
                count = len(hist_data)-1+a
                num = a
                a=0
                for i in range(num,count): 
                    industry_data.append([])
                    industry_data[-1].append(NowDate - timedelta(days=((91+5/16)*(i-num)))) #get exact date for quarter end
                        # Calculate the QuarterlyPriceGrowth and create target column
                    industry_data[-1].append(self.CalculateQuarterlyPriceChange(tick, industry_data[-1][0]))
                    # self.algorithm.Debug("Quarterly price change: "+industry[i][1])
                    industry_data[-1].append(self.CollectFeaturesForPrediction(tick, industry_data[-1][0]))
                    if not industry_data[-1][2]:
                        industry_data.pop()
                    else:
                        # self.algorithm.Debug("Price Change: "+str(industry_data[-1][1]))
                        # self.algorithm.Debug(f"Features Data for {tick} on {self.algorithm.Time}: {industry_data[-1][2]}")
                        a += 1
            else:
                self.algorithm.Debug(f"No hist_data collected in Ypred for {tick}")
        features = []
        target = []

        for i in range (0,len(industry_data)-1):
            if industry_data[i][1] and industry_data[i][2]:
                features.append(industry_data[i][2])
                target.append(industry_data[i][1])
                # features = industry_data[:][2]
                # target = industry_data[i][1]

        if features:
            if target:
                # Split the data into training and testing sets
                X = features # list of all features for all dates covered
                y = target # list of Quarterly Price Growth for all dates covered
                X = np.array(X)
                X = X.astype('float32')
                y = np.array(y)
                y = y.astype('float32')
                    # Define the neural network model
                model = keras.Sequential([
                    layers.Dense(32, activation='relu', input_shape=(10,)),
                    layers.Dense(16, activation='relu'),
                    layers.Dense(1, activation='linear')])  # Output layer, 1 neuron for regression

                    # Compile the model
                model.compile(optimizer='adam', loss='mean_squared_error')

                    # Train the model on the entire dataset
                model.fit(X, y, epochs=15, verbose=0)  # You can adjust the number of epochs

                    # Use the most recent data to make a prediction
                most_recent_data = X[-1].reshape(1, 10)
                prediction = model.predict(most_recent_data)[0][0]

                return prediction
            else:
                self.algorithm.Debug(f"Targets not collected for{tick} on {self.algorithm.Time}")
        else:
            self.algorithm.Debug(f"Features not collected for{tick} on {self.algorithm.Time}")
            return None
    
    def CollectFeaturesForPrediction(self, symbol, date):
            n = 1
            # sentence = tick.split()
            # symbol = sentence[0]
            # self.algorithm.Debug(f"Starting to collect features for {tick}")
            # self.algorithm.Debug(f"Symbol is: {symbol}")

            f1 = self.qb.GetFundamental(symbol, "OperationRatios.RevenueGrowth.OneYear", date-timedelta(n), date)
            if not f1.empty:
                f1 = f1.iloc[0][0]
                # self.algorithm.Debug(f"F1: {f1})")
            else:
                f1 = None
                # self.algorithm.Debug(f"f1 not found on {self.algorithm.Time}!")
            # self.algorithm.Log(a)
            f2 = self.qb.GetFundamental(symbol, "OperationRatios.NetIncomeGrowth.OneYear", date-timedelta(n), date)
            if not f2.empty:
                f2 = f2.iloc[0][0]
                # self.algorithm.Debug("F2: "+str(f2))
            else:
                f2 = None
                # self.algorithm.Debug("f2 not found!")
            # self.algorithm.Log(a)
            f3 = self.qb.GetFundamental(symbol, "OperationRatios.CurrentRatioGrowth.OneYear", date-timedelta(n), date)
            if not f3.empty:
                f3 = f3.iloc[0][0]
                # self.algorithm.Debug("F3: "+str(f3))
            else:
                f3= None
                # self.algorithm.Debug("f3 not found!")

            # self.algorithm.Log(a)
            f4 = self.qb.GetFundamental(symbol, "OperationRatios.LongTermDebtTotalCapitalRatio.OneYear", date-timedelta(n), date)
            if not f4.empty:
                f4 = f4.iloc[0][0]
                # self.algorithm.Debug("F4: "+str(f4))
            else:
                f4 = None
                # self.algorithm.Debug("f4 not found!")

            # self.algorithm.Log(a)
            f5 = self.qb.GetFundamental(symbol, "OperationRatios.TotalDebtEquityRatio.OneYear",date-timedelta(n), date)
            if not f5.empty:
                f5 = f5.iloc[0][0]
                # self.algorithm.Debug("F5: "+str(f5))
            else:
                f5 = None
                # self.algorithm.Debug("f5 not found!")

            # self.algorithm.Log(a)
            f6 = self.qb.GetFundamental(symbol, "EarningRatios.DilutedEPSGrowth.OneYear",date-timedelta(n), date)
            if not f6.empty:
                f6 = f6.iloc[0][0]
                # self.algorithm.Debug("F6: "+str(f6))
            else:
                f6 = None
                # self.algorithm.Debug("f6 not found!")

            # self.algorithm.Log(a)
            f7 = self.qb.GetFundamental(symbol, "EarningRatios.BookValuePerShareGrowth.OneYear", date-timedelta(n), date)
            if not f7.empty:
                f7 = f7.iloc[0][0]
                # self.algorithm.Debug("F7: "+str(f7))
            else:
                f7 = None
                # self.algorithm.Debug("f7 not found!")

            # self.algorithm.Log(a)
            f8 = self.qb.GetFundamental(symbol, "EarningRatios.FCFPerShareGrowth.OneYear",date-timedelta(n), date)
            if not f8.empty:
                f8 = f8.iloc[0][0]
                # self.algorithm.Debug("F8: "+str(f1))
            else:
                f8 = None
                # self.algorithm.Debug("f8 not found!")

            # self.algorithm.Log(a)
            f9 = self.qb.GetFundamental(symbol, "OperationRatios.NetMargin.OneYear", date-timedelta(n), date)
            if not f9.empty:
                f9 = f9.iloc[0][0]
                # self.algorithm.Debug("F9: "+str(f9))
            else:
                f9 = None
                # self.algorithm.Debug("f9 not found!")

            # self.algorithm.Log(a)
            if f1 and f2 and f3 and f4 and f5 and f6 and f7 and f8 and f9:
                end_date = date
                start_date = end_date - timedelta(days=14)
                if not self.algorithm.History(self.sym, start_date, end_date, Resolution.Daily).empty and not self.algorithm.History(symbol, start_date-timedelta(days=91), end_date-timedelta(days=91), Resolution.Daily).empty:
                    Price_data_now = self.algorithm.History(self.sym, start_date, end_date, Resolution.Daily)
                    nextEarningsData = self.algorithm.History(symbol, start_date-timedelta(days=91), end_date-timedelta(days=91), Resolution.Daily)
                    now_prices = statistics.mean(Price_data_now["close"])
                    nextPrices = statistics.mean(nextEarningsData["close"])
                    SP_PriceChange = (nextPrices - now_prices)/now_prices
                    if not SP_PriceChange:
                        return None
                        self.algorithm.Debug(f"SP_PriceMean not found for {symbol}")
                    else:
                        features = [f1,f2,f3,f4,f5,f6,f7,f8,f9, SP_PriceChange]
                        return features
                else:
                    return None
                    self.algorithm.Debug("No Price data for SPX")
            else:
                return None
                self.algorithm.Debug(f"Features not found for {symbol} on {date}")

    def CalculateQuarterlyPriceChange(self, symbol, date):
        # Fetch historical data and calculate the 
        if self.algorithm.Time > date + timedelta(days=90):
            end_date = date
            start_date = end_date - timedelta(days=14)
            Price_data_now = self.algorithm.History(symbol, start_date, end_date, Resolution.Daily)
            nextEarningsData = self.algorithm.History(symbol, start_date+timedelta(days=91), end_date+timedelta(days=91), Resolution.Daily)
            if not Price_data_now.empty and not nextEarningsData.empty:
                now_prices = statistics.mean(Price_data_now["close"])
                nextPrices = statistics.mean(nextEarningsData["close"])
                PercentDif = (nextPrices - now_prices)/now_prices
                return PercentDif
            else:
                return 0 
                self.algorithm.Debug(f"Quarterly Price Change for {symbol} on {date} was not obtained")
        else:
            return None
            self.algorithm.Debug("Quarterly Price not found because we are in this quarter")


class MyPortfolioConstructionModel(EqualWeightingPortfolioConstructionModel):

    def __init__(self, rebalance = Resolution.Daily):
        super().__init__(rebalance)
        self.Output = []
        self.current_price=0
        self.Preds = []

    def DetermineTargetPercent(self, activeInsights):
        # if self.algorithm.IsWarmingUp: 
        #     return
        # else:
            if activeInsights:
                for insight in activeInsights:
                    self.Preds.append(insight.Magnitude)

                total = self.CalculateTotalDesirability()

                for i in range (0,len(Symbols)-1):
                    # Fetch price data
                    symbol = Symbols[i]
                    current_price = symbol.Price

                    if not np.isnan(current_price) and symbol.Fundamentals.HasFundamentalData:
                            # Calculate days since the last quarterly earnings
                        days_since_earnings = self.CalculateDaysSinceQuarterly(symbol, self.algorithm.Time)

                            # Calculate original prices
                        original_prices = self.CalculateOriginalPrice(symbol)

                            # Calculate trading days in the quarter
                        trading_days_in_quarter = 91+5/16

                            # Check if expected growth is less than or equal to 0
                        Y = self.Preds[i]
                        if Y >= 0:
                            volatility = self.CalculateVolatility(symbol)
                            # self.algorithm.Debug(f"Volatility for {symbol}: {volatility}")
                                # Set a cap for allocation based on volatility
                            cap = 4 * volatility

                                # Check if percent distance is outside the cap range
                            percent_distance = self.CalculatePercentDistance(Y, trading_days_in_quarter, original_prices, current_price, days_since_earnings)/100
                            self.algorithm.Debug(f"Percent Distance for {symbol} on {self.algorithm.Time} is {percent_distance}")
                            # Calculate allocation based on expected growth and volatility
                            proportion_of_alloc = self.CalculateProportionOfAlloc(Y, volatility, percent_distance, total, symbol)
                            if percent_distance and Y and trading_dyas_in_quarter and original_prices and current_price and days_since_earnings and proportion_of_alloc:
                                if percent_distance < cap or percent_distance >= cap*2:
                                    self.Output[i] = 0  # Remove allocation
                                elif percent_distance > cap and percent_distance < cap*2:
                                    self.Output[i] = -(proportion_of_alloc)
                                else:
                                    self.Output[i] = proportion_of_alloc
                                
                                return self.Output
                                self.algorithm.Debug("self.Output was returned")
                        else:
                            self.algorithm.Debug(f"No prediction for {symbol}")
                    else:
                        self.algorithm.Debug(f"No current price found for {symbol}")
            else: 
                self.algorithm.Debug("No Active Insights Available")



    def OnSecuritiesChanged(self, algorithm, changes):
        self.algorithm = algorithm
        # ("Symbols list for Portfolio Construct: " + str(Symbols))
        
        super().OnSecuritiesChanged(algorithm, changes)
    def CalculatePercentDistance(self, y_pred, trading_days_in_quarter, original_prices, current_price, days_since_quarterly):
        expected_price = self.GenerateExpectedDailyPrice(y_pred, trading_days_in_quarter, original_prices, days_since_quarterly)
        # self.algorithm.Debug(f"expected_price: {expected_price}")
        if expected_price is not None:
            # self.algorithm.Debug(f"current_price: {current_price}")
            percent_distance = ((current_price - expected_price) / expected_price) * 100
            # self.algorithm.Debug(f"percent_distance: {percent_distance}")
            return percent_distance
        else:
            return 0
            self.algorithm.Debug("No expected price found!")

    def GenerateExpectedDailyPrice(self, y_pred, trading_days_in_quarter, original_price, days_since_quarterly):
        if y_pred > 0:
            # self.algorithm.Debug(f"y_pred: {y_pred}")
            # self.algorithm.Debug(f"trading_days: {trading_days_in_quarter}")
            daily_growth_rate = y_pred / trading_days_in_quarter
            # self.algorithm.Debug(f"daily growth: {daily_growth_rate}")
            # self.algorithm.Debug(f"originial_price: {original_price}")
            # self.algorithm.Debug(f"days since quarterly: {days_since_quarterly}")
            # self.algorithm.Debug(f"days_since_quarterly * daily_growth_rate: {days_since_quarterly * daily_growth_rate}")
            expected_price = original_price + days_since_quarterly * daily_growth_rate
            # self.algorithm.Debug(f"expected_price: {expected_price}")
            return expected_price
        else:
            return None

    def CalculateDaysSinceQuarterly(self, symbol, date):
        earningsDate = symbol.Fundamentals.EarningReports.FileDate.Value
        # self.algorithm.Debug(f"This is the earnings FileDate for(days since Q) {symbol}: {earningsDate}")

        days = date - earningsDate
        int(days.total_seconds())/3600
        if days is not None:
            return days
        else:
            return None
            self.algorithm.Debug(f"Days since quarterly not found for {symbol}!")
        
    def CalculateProportionOfAlloc(self, y_pred, volatility, percent_distance, total, symbol):
        desirability = y_pred * volatility * ((abs(percent_distance)**2)/100 + 1)

        if total > 0:
            proportion_of_alloc = desirability / total
            return proportion_of_alloc
        else:
            # self.algorithm.Debug(f'Proportion of Allocation not found for {symbol}(total !>0)')
            return None
    
    def CalculateTotalDesirability(self):
        desirability = []
        for i in range (0,len(Symbols)-1):
            # Fetch price data
            symbol_key = Symbols[i]
            current_price = symbol_key.Price

            if not np.isnan(current_price) and current_price>0:
                    # Calculate days since the last quarterly earnings
                days_since_earnings = self.CalculateDaysSinceQuarterly(symbol_key, self.algorithm.Time)

                    # Calculate original prices
                original_prices = self.CalculateOriginalPrice(symbol_key)

                    # Calculate trading days in the quarter
                trading_days_in_quarter = 91+5/16

                    # Check if expected growth is less than or equal to 0
                Y = self.Preds[i]
                if Y <= 0:
                    Y = 0

                volatility = self.CalculateVolatility(symbol_key)
                    # Set a cap for allocation based on volatility
                cap = 4 * volatility

                    # Check if percent distance is outside the cap range

                percent_distance = self.CalculatePercentDistance(Y, trading_days_in_quarter, original_prices, current_price, days_since_earnings)
                if (percent_distance/100) < -cap or (percent_distance/100)> 2*cap or percent_distance is None:
                    percent_distance = 0
                if percent_distance and Y:
                    d = Y * volatility * ((abs(percent_distance)**2)/100 + 1)
                    if d >= 0:
                        desirability.append(d)
                    else:
                        self.algorithm.Debug(f'Desirability not obtained for {symbol_key}')
                else:
                    self.algorithm.Debug(f"Details not found in total Desirability for {symbol_key}")
            else:
                self.algorithm.Debug(f"Current price not found for {symbol_key}")

        total = sum(desirability)
        if total is not None:
            return total
        else:
            return total
            self.algorithm.Debug(f"Total desirability not found for {symbol_key}")
                            
    def CalculateOriginalPrice(self, symbol_key):
            # Fetch historical price data for the specified date range
        symbol = symbol_key.Symbol
        last_earnings_release = symbol_key.Fundamentals.EarningReports.FileDate.Value

        # self.algorithm.Debug(f"Last earnings release for {self.algorithm.Time}: {last_earnings_release}")
        try:
            self.algorithm.History(symbol, last_earnings_release-timedelta(days=14), last_earnings_release).empty
            historical_data = self.algorithm.History(symbol, last_earnings_release-timedelta(days=14), last_earnings_release) #10 days of data
            # Calculate the average price
            average_price = statistics.mean(historical_data["close"])
            return average_price
        except:
            self.algorithm.Debug(f"No historical data available for {symbol_key} on {self.algorithm.Time}")
            return None

    def CalculateVolatility(self,symbol):
            # Get historical data for the last 100 days
        history = self.algorithm.History(symbol.Symbol, timedelta(days=200))

        if not history.empty:
                # Calculate daily returns
            close_prices = history["close"]
            daily_returns = close_prices.pct_change().dropna()

                # Calculate historical volatility (standard deviation of daily returns)
            volatility = np.std(daily_returns)
            if volatility:
                return volatility
            else:
                return 0
                self.algorithm.Debug(f"Volatility not found for {symbol} on {self.algorithm.Time}")

class MyAlgorithm(QCAlgorithm):
    def Initialize(self) -> None:
#         self.AddUniverse(self.SetUniverseSelection(QC500UniverseSelectionModel()))
        # self.AddUniverseSelection(LiquidValueUniverseSelectionModel())
        self.AddUniverse(self.MyFundamentalFunction)
        # tickers = ["SPY", "MSFT", "AAPL"]
        # symbols = [ Symbol.Create(ticker, SecurityType.Equity, Market.USA) for ticker in tickers]
        #self.Debug(self.spy)
        # self.symbol = self.AddEquity('AAPL', Resolution.Daily)
        # self.symbol = self.AddEquity('MSFT', Resolution.Daily)
        #self.AddEquity('SPY', Resolution.Daily)
        # self.AddUniverseSelection(ManualUniverseSelectionModel(symbols))
        #value = self.ActiveSecurities.Values["Symbol"][1]
        #self.Debug("ActiveSecurities.Values[1](ln404): "+ str(value))
        # fundamentals = self.Securities[self.symbol].Fundamentals
        # self.Debug(fundamentals)
        # self.Debug(str(self.ActiveSecurities.Values))
        # self.Debug("entering the loop for printing symbols")
        # for equity in self.ActiveSecurities.Values:
        #     self.Debug(equity)
        #     self.Debug("trying to find values")

        self.SetStartDate(2020, 11, 1)
        self.SetEndDate(2023, 11, 1)
        self.SetCash(100000)
        self.SetWarmUp(timedelta(days=120)) #enough to go through one quarter plus 2 weeks(needed to calculate original prices)
        self.UniverseSettings.Resolution = Resolution.Daily

        self.AddAlpha(NNAlphaModel(self))
        # self.Debug("Alpha Model ran through with no issues!")

        self.SetPortfolioConstruction(MyPortfolioConstructionModel(self))
        # self.Debug("Portfolio Construction ran through with no issues!")

        # self.AddRiskManagement(MaximumUnrealizedProfitPercentPerSecurity(0.1))
        # self.AddRiskManagement(MaximumDrawdownPercentPerSecurity(0.1))

        self.SetExecution(ImmediateExecutionModel())

        self.Debug("Model Officially Initialized!!!")

    # def OnData(self, data):
    #     pass
    def IsRebalanceDue(self):
        if self.Time.time() <time(16,0) and self.Time.time() > time(9,0):
            return True
        else:
            return False
    def MyFundamentalFunction(self, fundamental=List[Fundamental]):
        if self.IsRebalanceDue:
            filtered = [f for f in fundamental if f.Price > 1 and f.HasFundamentalData and f.DollarVolume > 1000 and f.EarningReports.BasicEPS.TwelveMonths<25 and f.AssetClassification.MorningstarSectorCode != MorningstarSectorCode.Healthcare and f.AssetClassification.MorningstarSectorCode != MorningstarSectorCode.Utilities]
            sortedByDollarVolume = sorted(filtered, key=lambda f: f.DollarVolume, reverse=True)[:50] #adjust for how many companies are wanted
            return [f.Symbol for f in sortedByDollarVolume]